In [4]:
import os, re, sys

# stage_in packages
from unity_sds_client.resources.collection import Collection

# stage_out packages
from datetime import datetime, timezone
from unity_sds_client.resources.dataset import Dataset
from unity_sds_client.resources.data_file import DataFile

# Import Files from STAC Item Collection

Load filenames from the stage_in STAC item collection file

In [5]:
input_stac_collection_file = 'test/stage_in/catalog_local.json' # type: stage-in
output_stac_catalog_dir    = 'process_results'                    # type: stage-out

output_collection = 'EMIT_TEST_FOR_JAY'

output_filename_prefix = 'test_output'

In [6]:
inp_collection = Collection.from_stac(input_stac_collection_file)
data_filenames = inp_collection.data_locations()

data_filenames

if2


['/home/gshen/test_data/EMIT_L1B/EMIT_L1B_RAD_001_20230620T084426_2317106_011.nc',
 '/home/gshen/test_data/EMIT_L1B/EMIT_L1B_OBS_001_20230620T084426_2317106_011.nc',
 '/home/gshen/test_data/EMIT_L2A/EMIT_L2A_RFL_001_20230620T084426_2317106_011.nc',
 '/home/gshen/test_data/EMIT_L2A/EMIT_L2A_RFLUNCERT_001_20230620T084426_2317106_011.nc',
 '/home/gshen/test_data/EMIT_L2A/EMIT_L2A_MASK_001_20230620T084426_2317106_011.nc']

# Grab input files for GHG process

Pass appropriate nc files into the ghg_process.py call

In [7]:
# need to add some robustness code and/or basic exception handling to deal with possibly mal-formed input catalog
r = re.compile('.*L1B_RAD*')
l1b_rad_filename = list(filter(r.match, data_filenames))[0]
r = re.compile('.*L1B_OBS*')
l1b_obs_filename = list(filter(r.match, data_filenames))[0]
r = re.compile('.*L2A_MASK*')
l2a_mask_filename = list(filter(r.match, data_filenames))[0]

#print(l1b_rad_filename)
#print(l1b_obs_filename)
#print(l2a_mask_filename)

# Create output directory
Create directory to store output files, it will be held locally at the path specified by the stage-out variable "output_stac_catalog_dir"

In [8]:
if not os.path.exists(output_stac_catalog_dir):
    os.system('mkdir ' + output_stac_catalog_dir)

# Run the GHG process
Pass appropriate nc files into the ghg_process.py call

In [9]:
sys.argv = ['ghg_process.py', l1b_rad_filename, l1b_obs_filename, l1b_rad_filename, 'blah.img', l2a_mask_filename, l2a_mask_filename, output_stac_catalog_dir+'/'+output_filename_prefix, '--state_subs', l2a_mask_filename, '--overwrite']
exec(open("./ghg_process.py").read())

/home/gshen/micromamba/envs/isofit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-02 09:17:14,368	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/gshen/micromamba/envs/isofit/lib/python3.10/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
INFO:2024-10-02,09:17:35 ||| starting parallel mf
INFO:2024-10-02,09:17:35 ||| Started processing input file: "/home/gshen/test_data/EMIT_L1B/EMIT_L1B_RAD_001_20230620T084426_2317106_011.nc"
INFO:2024-10-02,09:17:41 ||| Active wavelength range: [500.0, 2450.0]: 255 channels
INFO:2024-10-02,09:17:41 ||| load

# Create stage-out item catalog

In [12]:
# Create a collection
out_collection  = Collection(output_collection)

# Create Datasets for the MF output file
start_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat()
end_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat()
creation_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat()

mf_envi_file = output_filename_prefix + '_ch4_mf'
dataset = Dataset(
    name=mf_envi_file, 
    collection_id=out_collection.collection_id, 
    start_time=start_time, 
    end_time=end_time,
    creation_time=creation_time,
)
dataset.add_data_file(DataFile("data", mf_envi_file, ["data_envi"])) # Add output file(s) to the dataset

mf_envi_hdr = output_filename_prefix + '_ch4_mf.hdr'
dataset.add_data_file(DataFile("data", mf_envi_hdr, ["data_envi_hdr"])) # Add output file(s) to the dataset

catalog_target = output_filename_prefix + '_ch4_mf.json'
dataset.add_data_file(DataFile("json", catalog_target, ["metadata"])) # Add output file(s) to the dataset

# Add the dataset to the collection
#out_collection.add_dataset(dataset)
out_collection._datasets.append(dataset)

Collection.to_stac(out_collection, output_stac_catalog_dir)

In [13]:
out_collection.data_locations()

['test_output_ch4_mf', 'test_output_ch4_mf.hdr', 'test_output_ch4_mf.json']